In [1]:
from IPython.display import Math, display
import pandas as pd
import scipy as sp
import sympy as sym

In [2]:
# 多変量データ
fish_multi = pd.read_csv('fish_multi.csv')
fish_multi

,species,length
0,A,2
1,A,3
2,A,4
3,B,6
4,B,8
5,B,10


In [3]:
# 平均値を計算
group = fish_multi.groupby('species')
group.mean()

,length
species,
A,3
B,8


In [4]:
# 標準偏差を計算
group.std(ddof=1)

,length
species,
A,1.0
B,2.0


In [5]:
# 統計量を一気に計算
group.describe()

length                                    
         count mean  std  min  25%  50%  75%   max
species                                           
A          3.0  3.0  1.0  2.0  2.5  3.0  3.5   4.0
B          3.0  8.0  2.0  6.0  7.0  8.0  9.0  10.0

In [6]:
# クロス集計について見ていく
# 量的データと質的データが入り混じるデータはクロス集計する
shoes = pd.read_csv('shoes.csv')
shoes

,store,color,sales
0,tokyo,blue,10
1,tokyo,red,15
2,osaka,blue,13
3,osaka,red,9


In [7]:
# クロス集計する
cross = pd.pivot_table(
    data=shoes,
    values='sales',
    aggfunc='sum',
    index='store',
    columns='color'
)
cross

color,blue,red
store,,
osaka,13,9
tokyo,10,15


In [8]:
# 共分散
# 2つの量的変数の関係性を見るときに使われる統計量
# もし共分散Cov>0であれば、xが平均より大きい時はyも平均より大きいと、期待できる
x = sym.IndexedBase('x')
y = sym.IndexedBase('y')
μ = sym.IndexedBase('μ')
left = sym.Function('Cov')(x, y)
i, N = sym.symbols('i, N')
right = sym.Sum((x[i] - μ[x]) * (y[i] - μ[y]), (i, 1, N)) / N
equation = sym.Eq(left, right)
Math(sym.latex(equation))

<IPython.core.display.Math object>

In [9]:
# 不偏共分散
# 標本分散と同様、共分散を計算するデータが、母集団から抽出された標本である場合、バイアスを持つ
# 不偏性を担保するためn-1を使う
x = sym.IndexedBase('x')
y = sym.IndexedBase('y')
μ = sym.IndexedBase('μ')
left = sym.Function('Cov')(x, y)
i, N = sym.symbols('i, N')
right = sym.Sum((x[i] - μ[x]) * (y[i] - μ[y]), (i, 1, N)) / (N - 1)
equation = sym.Eq(left, right)
Math(sym.latex(equation))

<IPython.core.display.Math object>

In [10]:
# 分散共分散行列
# 複数の変数における分散と共分散の組み合わせを行列にしたもの
σ = sym.IndexedBase('σ')
x, y, z = sym.symbols('x, y, z')
left = sym.MatrixSymbol('Cov', 3, 3)
right0 = sym.Matrix([
    [σ[x, x]**2, σ[x, y]**2, σ[x, z]**2],
    [σ[y, x]**2, σ[y, y]**2, σ[y, z]**2],
    [σ[z, x]**2, σ[z, y]**2, σ[z, z]**2]
])
right1 = sym.Matrix([
    [σ[x]**2, sym.Function('Cov')(x, y), sym.Function('Cov')(x, z)],
    [sym.Function('Cov')(x, y), σ[y]**2, sym.Function('Cov')(x, y)],
    [sym.Function('Cov')(x, z), sym.Function('Cov')(x, y), σ[z]**2]
])
equation0 = sym.Eq(left, right0)
equation1 = sym.Eq(left, right1)
display(Math(sym.latex(equation0)))
display(Math(sym.latex(equation1)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [11]:
# 下記のデータの共分散を計算して見る
cov_data = pd.read_csv('cov.csv')
cov_data

,x,y
0,18.5,34
1,18.7,39
2,19.1,41
3,19.7,38
4,21.5,45
5,21.7,41
6,21.8,52
7,22.0,44
8,23.4,44
9,23.8,49


In [12]:
# 共分散を素朴に計算
x = cov_data['x']
y = cov_data['y']
N = len(cov_data)
mu_x = sp.mean(x)
mu_y = sp.mean(y)
cov_sample = sum((x - mu_x) * (y - mu_y)) / N
cov_sample

6.9060000000000015

In [13]:
# データが母集団から抽出された標本である場合の計算もする
cov = sum((x - mu_x) * (y - mu_y)) / (N - 1)
cov

7.673333333333336

In [14]:
# Scipyで分散共分散行列
sp.cov(x, y, ddof=0)

array([[ 3.2816,  6.906 ],
       [ 6.906 , 25.21  ]])

In [15]:
# 不偏分散共分散行列
sp.cov(x, y, ddof=1)

array([[ 3.64622222,  7.67333333],
       [ 7.67333333, 28.01111111]])

In [16]:
# ピアソンの積率相関係数
# いわゆる相関係数というやつ
# 共分散を-1~1の間に収まるように規格化したもの
ρ = sym.IndexedBase('ρ')
σ = sym.IndexedBase('σ')
x, y = sym.symbols('x, y')
left = ρ[x, y]
right = sym.Function('Cov')(x, y) / sym.sqrt(σ[x]**2 * σ[y]**2)
equation = sym.Eq(left, right)
Math(sym.latex(equation))

<IPython.core.display.Math object>

In [17]:
# 相関行列
# 複数の変数において相関係数の組み合わせを行列にまとめたもの
ρ = sym.IndexedBase('ρ')
x, y, z = sym.symbols('x, y, z')
left = sym.MatrixSymbol('Cov', 3, 3)
right = sym.Matrix([
    [1, ρ[x, y], ρ[x, z]],
    [ρ[y, x], 1, ρ[y, z]],
    [ρ[z, x]**2, ρ[z, y], 1]
])
equation = sym.Eq(left, right)
display(Math(sym.latex(equation)))

<IPython.core.display.Math object>

In [18]:
# ピアソンの積率相関係数、つまり相関係数を計算
x = cov_data['x']
y = cov_data['y']
sigma_2_x = sp.var(x, ddof=1)
sigma_2_y = sp.var(y, ddof=1)
rho = cov / sp.sqrt(sigma_2_x * sigma_2_y)
rho

0.7592719041137088

In [19]:
# 相関係数は不偏性を考慮しても相関係数は変わらない
# 分母と分子で同じ値が割られているため
sigma_2_x_sample = sp.var(x, ddof=0)
sigma_2_y_sample = sp.var(y, ddof=0)
cov_sample / sp.sqrt(sigma_2_x_sample * sigma_2_y_sample)

0.7592719041137087

In [20]:
# Scipyで相関行列
sp.corrcoef(x, y)

array([[1.       , 0.7592719],
       [0.7592719, 1.       ]])